In [1]:
import torch

### num_swapped_pairs

In [8]:
y_true = torch.tensor([3, 2, 1, 1, 3, 1, 2])
print(y_true)
y_pred = torch.tensor([7, 6, 5, 4, 3, 2, 1])
print(y_pred)

tensor([3, 2, 1, 1, 3, 1, 2])
tensor([7, 6, 5, 4, 3, 2, 1])


In [9]:
def num_swapped_pairs(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> torch.Tensor:
    """
    Function to calculate the number of incorrectly ordered 
    pairs (correct ordering is from the largest value in ys_true to the smallest) or rearranged pairs.
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values.
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.

    Returns:
        torch.Tensor: Number of incorrectly ordered or rearranged pairs.
    """
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    true_sorted_by_preds = torch.gather(ys_true, dim=-1, index=order)
    comb = torch.combinations(true_sorted_by_preds)
    return torch.sum(comb[:, 0] < comb[:, 1])

In [10]:
num_swapped_pairs(y_true, y_pred)

tensor(6)

### compute_gain

In [24]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    """
    Helper function for calculating DCG and NDCG, which calculates Gain.
    """
    if gain_scheme == "const":
        return y_value
    elif gain_scheme == "exp2":
        return 2**y_value - 1
    else:
        raise ValueError("gain function can be either const or exp2")

### DCG

In [25]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> torch.Tensor:
    """
    Function to calculate the DCG metric. 
    Discounted cumulative gain (DCG) is a measure of ranking quality.
    https://en.wikipedia.org/wiki/Discounted_cumulative_gain
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values.
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.
        gain_scheme (str): Gain scheme.

    Returns:
        torch.Tensor: DCG metric.
    """
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    true_sorted_by_preds = torch.gather(ys_true, dim=-1, index=order)
    
    gain_function = lambda x: compute_gain(x, gain_scheme)
    gains = gain_function(true_sorted_by_preds)
    
    discounts = torch.tensor(1) /  torch.log2(torch.arange(true_sorted_by_preds.shape[0], dtype=torch.double) + 2.0)
    discounted_gains = gains * discounts
    
    sum_dcg = torch.sum(discounted_gains, dim=-1)
    return sum_dcg

In [27]:
dcg(y_true, y_pred, "const")

tensor(7.3760, dtype=torch.float64)

### NDCG

In [30]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> torch.Tensor:
    """
    Function to calculate the NDCG metric. 
    Normalized discounted cumulative gain (NDCG) is normalized measure of ranking quality.
    https://en.wikipedia.org/wiki/Discounted_cumulative_gain
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values.
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.
        gain_scheme (str): Gain scheme.

    Returns:
        torch.Tensor: NDCG metric.
    """
    ideal_dcgs = dcg(ys_true, ys_true, gain_scheme)
    predicted_dcgs = dcg(ys_true, ys_pred, gain_scheme)
    ndcg_score = predicted_dcgs / ideal_dcgs
    return ndcg_score

In [31]:
ndcg(y_true, y_pred, "const")

tensor(0.9419, dtype=torch.float64)

### precision_at_k

In [39]:
y_true = torch.tensor([1, 0, 1, 0, 1, 0, 0])
print(y_true)
y_pred = torch.rand(7)
print(y_pred)

tensor([1, 0, 1, 0, 1, 0, 0])
tensor([0.3168, 0.6038, 0.3472, 0.2504, 0.9051, 0.7405, 0.6881])


In [40]:
def precision_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> torch.Tensor:
    """ 
    Function for calculating precision in the top-k positions for binary markup.
    
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values (positive is relevant position).
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.
        k (int): Number of top k positions.

    Returns:
        torch.Tensor: Precision at k metric.
    """
    if torch.sum(ys_true) == 0:
        return -1
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    true_sorted_by_preds = torch.gather(ys_true, dim=-1, index=order)
    return torch.sum(true_sorted_by_preds[:k]) / min(torch.sum(true_sorted_by_preds), k)

In [41]:
precision_at_k(y_true, y_pred, 5)

tensor(0.6667)

### average_precision

In [44]:
def average_precision(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> torch.Tensor:
    """
    Function for calculating average precision for binary markup.
    
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values (positive is relevant position).
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.

    Returns:
        torch.Tensor: Avearage precision metric.
    """
    if torch.sum(ys_true) == 0:
        return -1
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    true_sorted_by_preds = torch.gather(ys_true, dim=-1, index=order)
    rel_number_found_by_pos = torch.cumsum(true_sorted_by_preds, dim=0)
    pos_number = torch.arange(true_sorted_by_preds.shape[0], dtype=torch.double) + 1.0
    idx = true_sorted_by_preds == 1
    k = torch.sum(idx)
    return torch.sum(rel_number_found_by_pos[idx] / pos_number[idx], dim=0) / k

In [45]:
average_precision(y_true, y_pred)

tensor(0.6333, dtype=torch.float64)

### reciprocal_rank

In [46]:
y_true = torch.tensor([0, 0, 1, 0, 0, 0, 0])
print(y_true)
y_pred = torch.rand(7)
print(y_pred)

tensor([0, 0, 1, 0, 0, 0, 0])
tensor([0.1533, 0.8854, 0.8556, 0.4837, 0.8159, 0.7301, 0.0292])


In [47]:
def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    """
    Function for calculating MRR (without averaging, i.e. for one request and many documents).
    
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values (positive is relevant position).
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.

    Returns:
        torch.Tensor: Reciprocal rank.
    """
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    true_sorted_by_preds = torch.gather(ys_true, dim=-1, index=order)
    return 1/((true_sorted_by_preds == 1).nonzero().item() + 1)

In [48]:
reciprocal_rank(y_true, y_pred)

0.5

### p_found

In [49]:
y_true = torch.rand(7)
print(y_true)
y_pred = torch.rand(7)
print(y_pred)

tensor([0.1615, 0.8984, 0.6906, 0.2509, 0.1942, 0.5556, 0.3244])
tensor([0.8977, 0.9142, 0.2413, 0.2984, 0.7985, 0.2970, 0.7239])


In [52]:
def p_found(ys_true: torch.Tensor, ys_pred: torch.Tensor, p_break: float = 0.15) -> torch.Tensor:
    """
    P-found calculation function from Yandex. This function is usually used to assess the quality of ranking.
    https://catboost.ai/en/docs/references/pfound
    
    Args:
        ys_true (torch.Tensor): Tensor of actual relevancy values.
        ys_pred (torch.Tensor): Tensor of predicted relevancy values.

    Returns:
        torch.Tensor: P-found metric.
    """
    order = torch.argsort(ys_pred, descending=True, dim=-1)
    prel = torch.gather(ys_true, dim=-1, index=order)
    plook = [1]
    for i in range(1, len(prel)):
        plook.append(plook[i-1] * (1 - prel[i-1]) * (1 - p_break))
    return torch.sum(torch.tensor(plook) * prel)

In [53]:
p_found(y_true, y_pred)

tensor(0.9566)